In [3]:
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM
import os
import resampy

In [4]:
def load_data(data_path):
    data = []
    labels = []
    emotions = {'angry_05': 0,  'disgust_07': 1, 'fearful_06': 2, 'happy_03': 3, 'neutral_01': 4, 'sad_04': 5, 'suprised_08': 6} # Duygulara göre etiketler
    for emotion in emotions.keys():
        emotion_path = os.path.join(data_path, emotion)
        for filename in os.listdir(emotion_path):
            if filename.endswith('.wav'):
                filepath = os.path.join(emotion_path, filename)
                feature = extract_feature(filepath)
                data.append(feature)
                labels.append(emotions[emotion])
    return np.array(data), np.array(labels)

In [5]:
def extract_feature(file_path, n_mfcc=40, max_len=174):
    audio_data, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=n_mfcc)
    pad_width = max_len - mfccs.shape[1]
    if pad_width > 0:
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfccs = mfccs[:, :max_len]
    #print(mfccs)
    return mfccs

In [6]:
data, labels = load_data('C:/Users/Oztur/OneDrive/Desktop/website/Voice')
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [7]:
from keras.utils import to_categorical
print(y_train)
y_train_encoded = to_categorical(y_train, num_classes=7)
y_test_encoded = to_categorical(y_test, num_classes=7)
print(y_train_encoded)


[3 0 1 ... 3 3 2]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [8]:
x_train.shape

(1660, 40, 174)

In [9]:
cnn_model = Sequential()

# Convolutional ve MaxPooling katmanları
cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(40, 174, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='sigmoid'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
# Tam bağlantılı (fully connected) katmanlar
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(64, activation='sigmoid'))

# Çıkış katmanı
cnn_model.add(Dense(7, activation='softmax'))  # 10 sınıf için çıkış

c:\Users\Oztur\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
cnn_history = cnn_model.fit(x_train, y_train_encoded, batch_size=32, epochs=20, validation_data=(x_test, y_test_encoded))


Epoch 1/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 24s 309ms/step - accuracy: 0.2104 - loss: 1.9460 - val_accuracy: 0.2072 - val_loss: 1.8800
Epoch 2/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 17s 327ms/step - accuracy: 0.2152 - loss: 1.8581 - val_accuracy: 0.1855 - val_loss: 1.8434
Epoch 3/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 348ms/step - accuracy: 0.2491 - loss: 1.7924 - val_accuracy: 0.2964 - val_loss: 1.7750
Epoch 4/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 19s 367ms/step - accuracy: 0.3353 - loss: 1.6995 - val_accuracy: 0.3108 - val_loss: 1.6900
Epoch 5/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 18s 351ms/step - accuracy: 0.3975 - loss: 1.6027 - val_accuracy: 0.4627 - val_loss: 1.4741
Epoch 6/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 21s 396ms/step - accuracy: 0.5164 - loss: 1.4056 - val_accuracy: 0.5060 - val_loss: 1.3606
Epoch 7/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 381ms/step - accuracy: 0.6124 - loss: 1.1822 - val_accuracy: 0.5060 - val_loss: 1.2972
Epoch 8/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 20s 375ms/step - accuracy: 0.6479 - loss: 1.0840 - val_accu

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

plt.plot(cnn_history.history['loss'], label='Train Loss')
plt.plot(cnn_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [13]:
cnn_scores = cnn_model.evaluate(x_test, y_test_encoded, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1]*100))

CNN Model Accuracy: 72.05%


In [14]:
cnn_y_pred = cnn_model.predict(x_test)
cnn_y_pred_labels = np.argmax(cnn_y_pred, axis=1)
print(classification_report(y_test, cnn_y_pred_labels))

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
              precision    recall  f1-score   support

           0       0.72      0.86      0.78        76
           1       0.62      0.65      0.63        40
           2       0.75      0.61      0.67        76
           3       0.78      0.73      0.76        74
           4       0.75      0.65      0.70        37
           5       0.66      0.74      0.70        72
           6       0.76      0.78      0.77        40

    accuracy                           0.72       415
   macro avg       0.72      0.71      0.71       415
weighted avg       0.72      0.72      0.72       415



In [15]:
cnn_model.save('sesmodeli2.keras')